VERSION NOTES:
- Fama-MacBeth Statistical Significance tests

# Asset Pricing Tests

In [98]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from linearmodels.panel.model import FamaMacBeth
from linearmodels.asset_pricing import TradedFactorModel

## Data

In [63]:
ff_data = pd.read_csv('ff_data.csv', index_col=0, parse_dates=True)
factors = pd.read_csv('factors.csv', index_col=0, parse_dates=True)

In [64]:
# Test asset clean up
ff_data = ff_data.loc[factors.index[0]:factors.index[-1]] / 100

# Factor Models
ff3 = ff_data.iloc[:,0:3]
# ff5 = ff_data.iloc[:,0:5]
ff5_mom = pd.concat([ff5, ff_data.iloc[:,31]], axis=1)

# Test Asset Portfolios
size_book_assets = ff_data.iloc[:,6:31].apply(lambda x: x - ff_data['RF'])
industry_assets = ff_data.iloc[:,32:].apply(lambda x: x - ff_data['RF'])

# Macro Factors
factors = pd.concat([sm.add_constant(factors), ff_data['Mkt-RF']], axis=1)

## Analysis

Two sets of test assets will be used: 
1) 25 Size-Book Sorted Portfolios 
2) 30 Industry Portfolios

Several factor models will be tested (on each set of test assets):
1) Macro Factors
    - how well do they price the cross-section?
    - what are their risk premia if any?
2) Macro Factors + Market Factor
3) FF3
    - Compare Macro Factors to FF3
4) FF3 + Macro Factors
    - Given the Macro Factors are SMB + HML needed?
5) FF5
6) FF5 + Momentum
    - do the Macro Factors price better than the standard model?
7) FF5 + Momentum + Macro Factors
    - is the standard model subsumed by the Macro Factors?

### Linear Factor Model/SDF Approach

In [646]:
def factor_models(test_assets):
    '''
    Runs all factor models on the given set of test assets.
    Stores the model instances in a dictionary.
    '''
    
    model_results = {}
    
    # Macro Factors
    model1 = TradedFactorModel(test_assets, factors.iloc[:,1:3]).fit(cov_type='kernel')
    model_results['macro'] = model1
   
    # Macro Factors + Market
    model2 = TradedFactorModel(test_assets, factors.iloc[:,1:4]).fit(cov_type='kernel')
    model_results['macro+mkt'] = model2
   
    # FF3
    model3 = TradedFactorModel(test_assets, ff3).fit(cov_type='kernel')
    model_results['ff3'] = model3
    
    # FF3 + Macro Factors
    model4 = TradedFactorModel(test_assets, pd.concat([ff3, factors.iloc[:,1:3]], axis=1)).fit(cov_type='kernel')
    model_results['ff3+macro'] = model4
    
    # FF5 + Momentum
    model5 = TradedFactorModel(test_assets, ff5_mom).fit(cov_type='kernel')
    model_results['ff5+mom'] = model5
     
    # FF5 + Momentum + Macro Factors
    model6 = TradedFactorModel(test_assets, pd.concat([ff5_mom, factors.iloc[:,1:3]], axis=1)).fit(cov_type='kernel')
    model_results['ff5+macro'] = model6
    
    return model_results

#### Test Assets: 25 Size-Book Value Sorted Portfolios

In [647]:
size_book_models = factor_models(size_book_assets)

In [634]:
# dir(size_book_models['macro+mkt'])
# size_book_models['macro+mkt'].full_summary
# size_book_models['macro+mkt'].betas
# size_book_models['macro+mkt'].alphas

##### Macro Model

In [642]:
size_book_models['macro']

TradedFactorModel Estimation Summary                      
================================================================================
No. Test Portfolios:                 25   R-squared:                      0.0122
No. Factors:                          2   J-statistic:                    140.74
No. Observations:                   559   P-value                         0.0000
Date:                  Fri, Mar 17 2023   Distribution:                 chi2(25)
Time:                          18:21:52                                         
Cov. Estimator:                  kernel                                         
                                                                                
                            Risk Premia Estimates                             
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
gdp_factor     0.0018     0.0009     2.0540     0.0400   8.203e-05      0.0035
cpi_factor    -0.0006     0.0002    -2.8090     0.0050     -0.0011     -0.0002
==============================================================================

Covariance estimator:
KernelCovariance, Kernel: bartlett, Bandwidth: 18
See full_summary for complete results
LinearFactorModelResults, id: 0x1f410b73d00

In [643]:
size_book_models['macro+mkt']

TradedFactorModel Estimation Summary                      
================================================================================
No. Test Portfolios:                 25   R-squared:                      0.7295
No. Factors:                          3   J-statistic:                    78.617
No. Observations:                   559   P-value                         0.0000
Date:                  Fri, Mar 17 2023   Distribution:                 chi2(25)
Time:                          18:21:52                                         
Cov. Estimator:                  kernel                                         
                                                                                
                            Risk Premia Estimates                             
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
gdp_factor     0.0018     0.0011     1.7051     0.0882     -0.0003      0.0039
cpi_factor    -0.0006     0.0002    -3.1794     0.0015     -0.0010     -0.0002
Mkt-RF         0.0060     0.0020     3.0447     0.0023      0.0021      0.0099
==============================================================================

Covariance estimator:
KernelCovariance, Kernel: bartlett, Bandwidth: 8
See full_summary for complete results
LinearFactorModelResults, id: 0x1f410b697f0

When the market factor is added GDP's risk premia becomes insignificant, however CPI's premia is highly significant, but negative

In [644]:
size_book_models['macro+mkt']._jstat

J-statistic
H0: All alphas are 0
Statistic: 78.6173
P-value: 0.0000
Distributed: chi2(25)
WaldTestStatistic, id: 0x1f410b73f10

In [662]:
12*size_book_models['macro+mkt'].alphas.sort_values()

SMALL LoBM   -0.057937
ME2 BM1      -0.022097
ME3 BM1      -0.019921
BIG LoBM     -0.004123
ME5 BM4       0.000189
ME4 BM1       0.001224
ME5 BM2       0.011108
BIG HiBM      0.013852
ME4 BM2       0.015128
ME5 BM3       0.018175
ME4 BM3       0.018914
ME2 BM2       0.022258
ME3 BM3       0.022419
ME1 BM3       0.023439
ME1 BM2       0.023967
ME3 BM2       0.024074
ME4 BM4       0.026086
ME2 BM3       0.030310
ME4 BM5       0.032994
ME3 BM4       0.033622
ME2 BM5       0.038636
ME2 BM4       0.040327
ME3 BM5       0.047869
SMALL HiBM    0.055631
ME1 BM4       0.057163
Name: alpha, dtype: float64

Reject null that alphas are 0 for the Macro + Mkt Model; the simple model does not price the portfolios.

The J-statistic itself is the lowest amoung all the models (even FF5+Momentum+Macro). The biggest pricing errors are in the more extreme portfolios.

In [638]:
size_book_models['ff5+mom']._jstat

J-statistic
H0: All alphas are 0
Statistic: 88.1799
P-value: 0.0000
Distributed: chi2(25)
WaldTestStatistic, id: 0x1f4124ae520

FF5 + Momentum does not price the portfolios either.

In [652]:
size_book_models['ff3']

TradedFactorModel Estimation Summary                      
================================================================================
No. Test Portfolios:                 25   R-squared:                      0.9175
No. Factors:                          3   J-statistic:                    88.256
No. Observations:                   559   P-value                         0.0000
Date:                  Fri, Mar 17 2023   Distribution:                 chi2(25)
Time:                          18:24:55                                         
Cov. Estimator:                  kernel                                         
                                                                                
                            Risk Premia Estimates                             
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Mkt-RF         0.0060     0.0019     3.0906     0.0020      0.0022      0.0098
SMB            0.0023     0.0013     1.7471     0.0806     -0.0003      0.0048
HML            0.0029     0.0016     1.8897     0.0588     -0.0001      0.0060
==============================================================================

Covariance estimator:
KernelCovariance, Kernel: bartlett, Bandwidth: 15
See full_summary for complete results
LinearFactorModelResults, id: 0x1f4124d63a0

In [651]:
size_book_models['ff3+macro']

TradedFactorModel Estimation Summary                      
================================================================================
No. Test Portfolios:                 25   R-squared:                      0.9180
No. Factors:                          5   J-statistic:                    76.101
No. Observations:                   559   P-value                         0.0000
Date:                  Fri, Mar 17 2023   Distribution:                 chi2(25)
Time:                          18:24:56                                         
Cov. Estimator:                  kernel                                         
                                                                                
                            Risk Premia Estimates                             
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Mkt-RF         0.0060     0.0020     3.0569     0.0022      0.0022      0.0099
SMB            0.0023     0.0013     1.7741     0.0760     -0.0002      0.0048
HML            0.0029     0.0015     1.8978     0.0577  -9.615e-05      0.0060
gdp_factor     0.0018     0.0010     1.8593     0.0630  -9.701e-05      0.0037
cpi_factor    -0.0006     0.0002    -2.9389     0.0033     -0.0011     -0.0002
==============================================================================

Covariance estimator:
KernelCovariance, Kernel: bartlett, Bandwidth: 13
See full_summary for complete results
LinearFactorModelResults, id: 0x1f41212d7c0

Macro factors do not subsume SMB and HML, however GDP is significant at 6.3% while CPI is highly significant.

In [653]:
size_book_models['ff5+macro']

TradedFactorModel Estimation Summary                      
================================================================================
No. Test Portfolios:                 25   R-squared:                      0.9247
No. Factors:                          8   J-statistic:                    81.472
No. Observations:                   559   P-value                         0.0000
Date:                  Fri, Mar 17 2023   Distribution:                 chi2(25)
Time:                          18:24:56                                         
Cov. Estimator:                  kernel                                         
                                                                                
                            Risk Premia Estimates                             
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Mkt-RF         0.0060     0.0019     3.0906     0.0020      0.0022      0.0098
SMB            0.0023     0.0013     1.7471     0.0806     -0.0003      0.0048
HML            0.0029     0.0016     1.8897     0.0588     -0.0001      0.0060
RMW            0.0027     0.0012     2.3289     0.0199      0.0004      0.0050
CMA            0.0031     0.0010     2.9306     0.0034      0.0010      0.0051
Mom            0.0060     0.0018     3.2642     0.0011      0.0024      0.0097
gdp_factor     0.0018     0.0009     1.9244     0.0543  -3.311e-05      0.0036
cpi_factor    -0.0006     0.0002    -2.8745     0.0040     -0.0011     -0.0002
==============================================================================

Covariance estimator:
KernelCovariance, Kernel: bartlett, Bandwidth: 15
See full_summary for complete results
LinearFactorModelResults, id: 0x1f410b7d610

GDP and CPI are not subsummed by FF5 + Macro; GDP's significance improves slightly.

In [648]:
size_book_models['ff5+macro']._jstat

J-statistic
H0: All alphas are 0
Statistic: 81.4717
P-value: 0.0000
Distributed: chi2(25)
WaldTestStatistic, id: 0x1f410b73d60

FF5 + Momentum + Macro does not price the portfolios either.

#### Test Assets: 30 Industry Portfolios

In [654]:
industry_models = factor_models(industry_assets)

In [657]:
industry_models['macro+mkt']

TradedFactorModel Estimation Summary                      
================================================================================
No. Test Portfolios:                 30   R-squared:                      0.5565
No. Factors:                          3   J-statistic:                    75.662
No. Observations:                   559   P-value                         0.0000
Date:                  Fri, Mar 17 2023   Distribution:                 chi2(30)
Time:                          18:30:32                                         
Cov. Estimator:                  kernel                                         
                                                                                
                            Risk Premia Estimates                             
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
gdp_factor     0.0018     0.0010     1.8217     0.0685     -0.0001      0.0037
cpi_factor    -0.0006     0.0002    -2.9842     0.0028     -0.0011     -0.0002
Mkt-RF         0.0060     0.0020     3.0537     0.0023      0.0022      0.0099
==============================================================================

Covariance estimator:
KernelCovariance, Kernel: bartlett, Bandwidth: 12
See full_summary for complete results
LinearFactorModelResults, id: 0x1f410b70f40

In [655]:
industry_models['ff3+macro']

TradedFactorModel Estimation Summary                      
================================================================================
No. Test Portfolios:                 30   R-squared:                      0.5935
No. Factors:                          5   J-statistic:                    82.979
No. Observations:                   559   P-value                         0.0000
Date:                  Fri, Mar 17 2023   Distribution:                 chi2(30)
Time:                          18:30:32                                         
Cov. Estimator:                  kernel                                         
                                                                                
                            Risk Premia Estimates                             
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Mkt-RF         0.0060     0.0020     3.0473     0.0023      0.0021      0.0099
SMB            0.0023     0.0012     1.8611     0.0627     -0.0001      0.0046
HML            0.0029     0.0015     1.9815     0.0475   3.195e-05      0.0058
gdp_factor     0.0018     0.0011     1.6688     0.0952     -0.0003      0.0039
cpi_factor    -0.0006     0.0002    -3.2214     0.0013     -0.0010     -0.0003
==============================================================================

Covariance estimator:
KernelCovariance, Kernel: bartlett, Bandwidth: 7
See full_summary for complete results
LinearFactorModelResults, id: 0x1f4124619a0

GDP is not significant.

In [658]:
industry_models['ff5+macro']

TradedFactorModel Estimation Summary                      
================================================================================
No. Test Portfolios:                 30   R-squared:                      0.6246
No. Factors:                          8   J-statistic:                    107.16
No. Observations:                   559   P-value                         0.0000
Date:                  Fri, Mar 17 2023   Distribution:                 chi2(30)
Time:                          18:30:32                                         
Cov. Estimator:                  kernel                                         
                                                                                
                            Risk Premia Estimates                             
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Mkt-RF         0.0060     0.0020     3.0559     0.0022      0.0022      0.0099
SMB            0.0023     0.0012     1.8400     0.0658     -0.0001      0.0047
HML            0.0029     0.0015     1.9493     0.0513    -1.6e-05      0.0059
RMW            0.0027     0.0011     2.4034     0.0162      0.0005      0.0050
CMA            0.0031     0.0010     3.1261     0.0018      0.0011      0.0050
Mom            0.0060     0.0019     3.1603     0.0016      0.0023      0.0098
gdp_factor     0.0018     0.0010     1.7302     0.0836     -0.0002      0.0038
cpi_factor    -0.0006     0.0002    -3.1304     0.0017     -0.0010     -0.0002
==============================================================================

Covariance estimator:
KernelCovariance, Kernel: bartlett, Bandwidth: 9
See full_summary for complete results
LinearFactorModelResults, id: 0x1f410ca0550

CPI is still highly significant, while GDP is not. The J-stat is quite high as well.

### Fama-MacBeth

#### Full-Sample

##### 1st Stage

In [260]:
beta = []
for portfolio in size_book_assets:
    ts = sm.OLS(size_book_assets.loc[size_book_assets.index,portfolio],factors.iloc[:,0:3]).fit()
    beta.append(ts.params[1:])

beta = pd.DataFrame(beta)
beta.index = size_book_assets.columns

##### 2nd Stage

In [261]:
lambdas = []
alphas = []
for t in range(0,size_book_assets.index.shape[0]):
    cs = sm.OLS(np.array(size_book_assets)[t].T, betas).fit()
    lambdas.append(cs.params)
    alphas.append(cs.resid)

In [404]:
lambdas_df = pd.DataFrame(lambdas)
lambdas_df.mean()

const         0.901521
gdp_factor    0.000045
cpi_factor   -0.001267
dtype: float64

In [412]:
se_gdp = ((lambdas_df['gdp_factor'] - lambdas_df.mean()['gdp_factor'])**2).sum() / len(lambdas_df)**2

In [416]:
# Fama-MacBeth t-stat without controlling for autocorrelation 
print(f'{lambdas_df.mean()["gdp_factor"]/se_gdp:0.5f}')

3.06668


In [505]:
# Joint test of alphas
alphas_df = pd.DataFrame(alphas)
alpha_hat = alphas_df.mean()

In [580]:
cov_alpha = ((alphas_df - alpha_hat).T @ (alphas_df - alpha_hat)) / len(alphas_df)**2

In [581]:
# Chi Squared Stat
alpha_chi = alpha_hat.T @ np.linalg.inv(cov_alpha) @ alpha_hat.T
alpha_chi

7.630754656593366

In [568]:
from scipy.stats import chi2

In [577]:
alpha_p_val = 1 - chi2(len(alphas_df)-1).cdf(alpha_chi)
alpha_p_val

1.0

#### 5 year Rolling Window

##### 1st Stage

In [264]:
from statsmodels.regression.rolling import RollingOLS

In [395]:
beta = {}
for portfolio in size_book_assets:
    ts = RollingOLS(size_book_assets.loc[size_book_assets.index,portfolio],factors.iloc[:,0:3], window=60).fit()
    beta[portfolio] = ts.params[1:]

In [396]:
betas_T = {}
for t in range(60,len(size_book_assets)-60):
    betas = []
    for portfolio in size_book_assets.columns:
        betas.append(beta[portfolio].iloc[t])
    
    betas_df = pd.DataFrame(np.array(betas), index=size_book_assets.columns, columns=['const', 'gdp_factor', 'cpi_factor'])
    
    betas_T[t] = betas_df

##### 2nd Stage

In [397]:
lambdas = []
alphas = []
for t in range(60,len(size_book_assets)-60):
    cs = sm.OLS(np.array(size_book_assets)[t].T, betas_T[t]).fit()
    lambdas.append(cs.params)
    alphas.append(cs.resid)

In [398]:
pd.DataFrame(lambdas).mean()

const         0.901521
gdp_factor    0.000045
cpi_factor   -0.001267
dtype: float64

In [401]:
# pd.DataFrame(alphas).mean()

### GMM?